# Modifying and deleting data

Sometimes you might want to modify some metadata after running the data through the standardisation scripts.
After the standardisation process the metadata associated with some data can still be edited. This can save
time if the data standardisation process is quite time consuming. Data can also be deleted from the object store.

In [2]:
from openghg.store import data_handler_lookup
from openghg.tutorial import populate_footprint_inert, clear_tutorial_store, tutorial_store_path

ImportError: cannot import name 'clear_tutorial_store' from 'openghg.tutorial._tutorial' (/Users/gar/Documents/Devel/openghg/openghg/tutorial/_tutorial.py)

In [ ]:
tutorial_store_path()

We'll first add some footprint data to the object store.

In [ ]:
clear_tutorial_store()

In [ ]:
populate_footprint_inert()

In [ ]:
result = data_handler_lookup(data_type="footprints", site="TAC", height="100m")

In [ ]:
result.metadata

We want to update the model name so we'll use the ``update_metadata`` method of the ``DataHandler`` object. To do this we need to take the
UUID of the Datasource returned by the ``data_handler_lookup`` function, this is the key of the metadata dictionary.

> **_NOTE:_**  The UUID below will be different on your computer. Take the UUID from the metadata dictionary.

In [ ]:
uuid = "60d68d7b-7d13-4b1d-8c78-237f9f7a0dea"

In [ ]:
updated = {"model": "new_model"}

result.update_metadata(uuid=uuid, to_update=updated)

When you run `update_metadata` the internal store of metadata for each `Datasource` is updated. If you want to **really** make sure that the metadata in the object store has been updated you can run `refresh`.

In [ ]:
result.refresh()

In [ ]:
metadata = result.metadata[uuid]

And check the model has been changed.

In [ ]:
metadata["model"]

## Deleting keys

Let's accidentally add too much metadata for the footprint and then delete.

In [ ]:
excess_metadata = {"useless_key": "useless_value"}
new_result.update_metadata(uuid=uuid, to_update=excess_metadata)

Oh no! We've added some useless metadata, let's remove it.

In [ ]:
to_delete = ["useless_key"]
new_result.update_metadata(uuid=uuid, to_delete=to_delete)

In [ ]:
result = data_handler_lookup(data_type="footprints", site="TAC", height="100m")

And check if the key is in the metadata:

In [ ]:
"useless_key" in result.metadata[uuid]

# Restore from backup

If you've accidentally pushed some bad metadata you can fix this easily by restoring from backup. Each `DataHandler` object stores a backup of the current metadata each time you run `update_metadata`. Let's add some bad metadata, have a quick look at the backup and then restore it.

In [ ]:
result = data_handler_lookup(data_type="footprints", site="TAC", height="100m")

In [ ]:
result.metadata.keys()

In [ ]:
uuid = "b703e490-2fdd-4bb3-bb16-66673673bf16"

In [ ]:
bad_metadata = {"domain": "neptune"}

In [ ]:
result.update_metadata(uuid=uuid, to_update=bad_metadata)

In [ ]:
result.metadata[uuid]["domain"]

In [ ]:
result.view_backup()

In [ ]:
result.restore(uuid=uuid)

In [ ]:
result.metadata[uuid]["domain"]

In [ ]:
result.refresh()

In [ ]:
result.metadata[uuid]["domain"]

## Multiple backups

In [ ]:
more_metadata = {"time_period": "1m"}
result.update_metadata(uuid=uuid, to_update=more_metadata)

In [ ]:
result.view_backup(uuid=uuid, version=2)

# Deleting data

To remove data from the object store we use `data_handler_lookup` again

In [ ]:
result = data_handler_lookup(data_type="footprints", site="TAC", height="100m")

In [ ]:
result.metadata

Each key of the metadata dictionary is a Datasource UUID. Please make sure that you double check the UUID of the Datasource you want to delete, this operation cannot be undone! Again, remember to change the UUID.

In [ ]:
uuid = "b2177d42-9df9-4a08-b50b-8aadbd7fb9d7"

In [ ]:
result.delete_datasource(uuid=uuid)

To make sure it's gone let's run the search again

In [ ]:
result = data_handler_lookup(data_type="footprints", site="TAC", height="100m")

In [ ]:
result.metadata

An empty dictionary means no results, the deletion worked.